In [1]:
import csv
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# import from scripts
import os
current_wd = os.getcwd()
os.chdir(os.path.abspath("..\\..\\..\\isttc\\scripts"))
from cfg_global import project_folder_path
os.chdir(current_wd)

In [2]:
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
plt.rcParams['svg.fonttype'] = 'none'

In [3]:
dataset_folder = project_folder_path + 'results\\allen_mice\\dataset\\'
fig_folder = project_folder_path + 'results\\allen_mice\\fig_draft_paper\\'

In [4]:
# 8 areas visual (V1, LM, AL, RL, AM, PM) and thalamic (LGN, LP)
# below mapping from Rudelt at el (found in code)
#    structures = ["VISp", "VISl", "VISrl", "VISal", "VISpm", "VISam", "LGd", "LP"]
#    structures_names = ["V1", "LM", "RL", "AL", "PM", "AM", "LGN", "LP"]
brain_areas = ['VISp', 'VISl', 'VISal', 'VISrl', 'VISam', 'VISpm', 'LGd', 'LP']

### Get data

In [5]:
csv_data_file = dataset_folder + 'cut_30min\\sua_list.csv'
with open(csv_data_file, newline='') as f:
    reader = csv.reader(f)
    sua_list = list(reader)
print(f'Loaded N units {len(sua_list)}')

Loaded N units 18168


In [6]:
column_names = ["specimen_id", "session_id", "unit_id", "ecephys_structure_acronym", 'firing_rate', 'amplitude_cutoff', 'isi_violations', 'presence_ratio']
units_df = pd.DataFrame([row[:8] for row in sua_list], columns=column_names)

In [7]:
units_subset_df = units_df[units_df['ecephys_structure_acronym'].isin(brain_areas)].copy()
print(f'N units from 8 areas {len(units_subset_df)}')

N units from 8 areas 9279


In [8]:
units_subset_df.head()

,specimen_id,session_id,unit_id,ecephys_structure_acronym,firing_rate,amplitude_cutoff,isi_violations,presence_ratio
66,744912849,766640955,950913540,VISam,5.92483027278476,0.00495112650974547,0.157932742873785,0.99
67,744912849,766640955,950915005,VISam,4.54907168784514,0.00120262444599965,0.09607244219631,0.99
68,744912849,766640955,950915018,VISam,1.20234792323596,0.00774320455622096,0.0528945259554864,0.99
69,744912849,766640955,950913754,VISam,2.72346544519189,0.0859747406874157,0.087628672265043,0.98
70,744912849,766640955,950913721,VISam,2.18650443154006,0.0610517885798365,0.0719751923948018,0.99


In [11]:
units_subset_df['firing_rate'] = units_subset_df['firing_rate'].astype(float)
units_subset_df['amplitude_cutoff'] = units_subset_df['amplitude_cutoff'].astype(float)
units_subset_df['isi_violations'] = units_subset_df['isi_violations'].astype(float)
units_subset_df['presence_ratio'] = units_subset_df['presence_ratio'].astype(float)

In [18]:
units_subset_df_quality = units_subset_df.query('amplitude_cutoff < 0.01 and isi_violations < 0.5 and presence_ratio > 0.9').copy()
print(f'N units from 8 areas {len(units_subset_df_quality)}')

N units from 8 areas 5775


In [19]:
units_subset_df_quality.groupby(by='ecephys_structure_acronym', as_index=False).count()

,ecephys_structure_acronym,specimen_id,session_id,unit_id,firing_rate,amplitude_cutoff,isi_violations,presence_ratio
0,LGd,254,254,254,254,254,254,254
1,LP,599,599,599,599,599,599,599
2,VISal,928,928,928,928,928,928,928
3,VISam,912,912,912,912,912,912,912
4,VISl,727,727,727,727,727,727,727
5,VISp,1063,1063,1063,1063,1063,1063,1063
6,VISpm,586,586,586,586,586,586,586
7,VISrl,706,706,706,706,706,706,706


In [16]:
units_subset_df.groupby(by='ecephys_structure_acronym', as_index=False).count()

,ecephys_structure_acronym,specimen_id,session_id,unit_id,firing_rate,amplitude_cutoff,isi_violations,presence_ratio
0,LGd,407,407,407,407,407,407,407
1,LP,1044,1044,1044,1044,1044,1044,1044
2,VISal,1483,1483,1483,1483,1483,1483,1483
3,VISam,1453,1453,1453,1453,1453,1453,1453
4,VISl,1142,1142,1142,1142,1142,1142,1142
5,VISp,1679,1679,1679,1679,1679,1679,1679
6,VISpm,919,919,919,919,919,919,919
7,VISrl,1152,1152,1152,1152,1152,1152,1152
